## Functional API

In [1]:
from tensorflow.keras.datasets import california_housing

(X_train, y_train), (X_test, y_test) = california_housing.load_data()

743530/743530 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Normalization, Input, Concatenate

input_ = Input(shape=[8], name='input')
norm_layer = Normalization(name='normalization')
hidden_layer1 = Dense(16, activation='relu', name='hidden1')
hidden_layer2 = Dense(16, activation='relu', name='hidden2')
hidden_layer3 = Dense(24, activation='relu', name='hidden3')
concat_layer1 = Concatenate(name='concat1')
concat_layer2 = Concatenate(name='concat2')
output_layer = Dense(1, name='output')

norm = norm_layer(input_)
hidden1 = hidden_layer1(norm)
hidden2 = hidden_layer2(hidden1)
concat1 = concat_layer1([input_, hidden2])
hidden3 = hidden_layer3(input_)
concat2 = concat_layer2([hidden3, concat1])
output = output_layer(concat2)

model = Model(inputs=[input_], outputs=[output])

In [ ]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 8)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 8)         │         17 │ input[0][0]       │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden1 (Dense)     │ (None, 16)        │        144 │ normalization[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden2 (Dense)     │ (None, 16)        │        272 │ hidden1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden3 (Dense)     │ (None, 24)        │        216 │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat1             │ (None, 24)        │          0 │ input[0][0],      │
│ (Concatenate)       │                   │            │ hidden2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat2             │ (None, 48)        │          0 │ hidden3[0][0],    │
│ (Concatenate)       │                   │            │ concat1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         49 │ concat2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 698 (2.73 KB)

 Trainable params: 681 (2.66 KB)

 Non-trainable params: 17 (72.00 B)

In [ ]:
model.save('deep_and_wide_model_v2.keras')

## Subclass API

In [46]:
from tensorflow.keras.models import Model
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.layers import Dense, Normalization, Input, Concatenate

@register_keras_serializable()
class MySubclassAPI(Model):
  def __init__(self, shape, units=16, activation='relu', **kwargs):
    super().__init__(**kwargs)
    self.norm_layer = Normalization()
    self.hidden_layer1 = Dense(units, activation=activation, name='hidden1')
    self.hidden_layer2 = Dense(units, activation=activation, name='hidden2')
    self.hidden_layer3 = Dense(24, activation=activation, name='hidden3')
    self.concat_layer1 = Concatenate(name='concat1')
    self.concat_layer2 = Concatenate(name='concat2')
    self.output_layer = Dense(1, name='output')

  def call(self, X):
    norm = self.norm_layer(X)
    hidden1 = self.hidden_layer1(norm)
    hidden2 = self.hidden_layer2(hidden1)
    concat1 = self.concat_layer1([X, hidden2])
    hidden3 = self.hidden_layer3(X)
    concat2 = self.concat_layer2([hidden3, concat1])
    output = self.output_layer(concat2)
    return output

In [47]:
from tensorflow.keras.optimizers import SGD, Adam

sub_model = MySubclassAPI(shape=8, units=16)
sub_model.norm_layer.adapt(X_train)
optimizer = Adam(learning_rate=0.001)
sub_model.compile(optimizer=optimizer, loss='mse', metrics=['RootMeanSquaredError'])

In [48]:
sub_model.fit(X_train, y_train, epochs=100, validation_split=0.2)

Epoch 1/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - RootMeanSquaredError: 232787.5156 - loss: 54211993600.0000 - val_RootMeanSquaredError: 211594.5000 - val_loss: 44772233216.0000
Epoch 2/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 201188.6094 - loss: 40533897216.0000 - val_RootMeanSquaredError: 174896.0938 - val_loss: 30588645376.0000
Epoch 3/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - RootMeanSquaredError: 169426.6406 - loss: 28710397952.0000 - val_RootMeanSquaredError: 165399.4688 - val_loss: 27356983296.0000
Epoch 4/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - RootMeanSquaredError: 160244.8594 - loss: 25683120128.0000 - val_RootMeanSquaredError: 161664.2031 - val_loss: 26135316480.0000
Epoch 5/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 156849.2812 - loss: 24605552640.0000 - val_RootMeanSquaredError: 156478.5469 - val_loss: 24485535744.0000
Epoch 6/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 151705.7

In [ ]:
sub_model.save("drive/MyDrive/MySubmodelAPI.keras")

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


## Test Custom Model

In [49]:
y_pred = sub_model.predict(X_test).reshape(y_test.shape[0])

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [69]:
y_pred[:5], y_test[:5]

(array([226671.95, 299055.06, 185373.84, 324483.12, 219379.69],
       dtype=float32),
 array([397900., 227900., 172100., 186500., 148900.], dtype=float32))

## Simple Verion of the Model

In [60]:
from tensorflow.keras.models import Model
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.layers import Dense, Normalization, Input, Concatenate, Dropout

@register_keras_serializable()
class MySubclassAPI(Model):
  def __init__(self, shape, units=16, activation='relu', **kwargs):
    super().__init__(**kwargs)
    self.norm_layer = Normalization()
    self.hidden_layer1 = Dense(units, activation=activation, name='hidden1')
    self.hidden_layer2 = Dense(units, activation=activation, name='hidden2')
    self.hidden_layer3 = Dense(units, activation=activation, name='hidden3')
    self.dropout = Dropout(0.2)
    self.output_layer = Dense(1, name='output')

  def call(self, X):
    norm = self.norm_layer(X)
    hidden1 = self.hidden_layer1(norm)
    hidden2 = self.hidden_layer2(hidden1)
    hidden3 = self.hidden_layer3(hidden2)
    output = self.output_layer(hidden3)
    return output

In [61]:
from tensorflow.keras.optimizers import SGD, Adam

sub_model = MySubclassAPI(shape=8, units=32)
sub_model.norm_layer.adapt(X_train)
optimizer = Adam(learning_rate=0.0005)
sub_model.compile(optimizer=optimizer, loss='mse', metrics=['RootMeanSquaredError'])

In [62]:
sub_model.fit(X_train, y_train, epochs=100, validation_split=0.2)

Epoch 1/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - RootMeanSquaredError: 234999.5156 - loss: 55228813312.0000 - val_RootMeanSquaredError: 236719.5625 - val_loss: 56036151296.0000
Epoch 2/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 231962.2344 - loss: 53822496768.0000 - val_RootMeanSquaredError: 216709.9688 - val_loss: 46963212288.0000
Epoch 3/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 203447.3750 - loss: 41443876864.0000 - val_RootMeanSquaredError: 161020.6250 - val_loss: 25927643136.0000
Epoch 4/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 144687.9219 - loss: 20962955264.0000 - val_RootMeanSquaredError: 120040.0312 - val_loss: 14409608192.0000
Epoch 5/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 116792.3516 - loss: 13645087744.0000 - val_RootMeanSquaredError: 109199.9453 - val_loss: 11924627456.0000
Epoch 6/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 107467.7

In [67]:
y_pred = sub_model.predict(X_test).reshape(y_test.shape[0])

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [68]:
y_pred[5:10], y_test[5:10]

(array([ 78357.8 , 293739.66, 248846.02, 246950.75, 162222.92],
       dtype=float32),
 array([ 65600., 344600., 235600., 233000., 192100.], dtype=float32))